# Patch Extraction from WSI

Extract and save a grid of patches at level 0 (highest resolution).

In [ ]:
import sys, os
from pathlib import Path
try:
    from shared import utils as u
except ImportError:
    repo_url = "https://github.com/anand-indx/dp-t25.git"; dest = "/content/dp-t25"
    if 'google.colab' in sys.modules and not os.path.exists(dest):
        import subprocess
        subprocess.run(['git', 'clone', '--depth', '1', repo_url, dest], check=False)
        sys.path.insert(0, dest)
    else:
        sys.path.insert(0, str(Path.cwd().parents[1]))
    from shared import utils as u

from openslide import OpenSlide, OpenSlideError
from pathlib import Path

DATA_DIR = u.get_data_dir()
WSI_PATH = u.get_wsi_path(DATA_DIR)
PATCH_DIR = DATA_DIR / 'patches'
PATCH_DIR.mkdir(parents=True, exist_ok=True)

tile_size, stride = 256, 256

try:
    slide = OpenSlide(str(WSI_PATH))
except OpenSlideError as e:
    raise RuntimeError(f"Cannot open WSI at {WSI_PATH}. Upload a file there or set WSI_PATH.")

for y in range(0, slide.dimensions[1]-tile_size+1, stride):
    for x in range(0, slide.dimensions[0]-tile_size+1, stride):
        img = slide.read_region((x, y), 0, (tile_size, tile_size)).convert('RGB')
        img.save(PATCH_DIR / f'{x}_{y}.png')
slide.close(); print(f'Done. Saved patches to {PATCH_DIR}')